In [1]:
import pandas as pd
import numpy as np

In [135]:
df = pd.read_csv("answers.csv", index_col=0)
# El salario de las personas de Mx se guarda en la columna salarymx y está en pesos.
# Si el valor de salarymx es == 0 es porque son de otro país y su salario está en la columna salaryusd (en dólares). 
df = df[(df['salarymx'] > 0)]
df["salarymx"].count()

994

In [8]:
# Agrupamos en base a años de experiencia.
exp = df.groupby("experience")["salarymx"].agg(['median', 'count'])
# Solamente tomamos en cuenta los grupos en los que tenemos por lo menos 5 observaciones
exp = exp[(exp['count'] > 4)]
exp.head()

,median,count
experience,,
0,18250,8
1,10000,53
2,18000,72
3,25000,57
4,25000,70


In [43]:
# Importamos lo que necesitamos para generar graficas con Bokeh
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, HoverTool, NumeralTickFormatter

output_notebook()

Loading BokehJS ...

In [131]:
# Si usamos directamente el num de observaciones para el tamaño del círculo nos da valores muy altos
# así que calculamos un valor amortiguado y lo asignamos a una nueva columna 'size'.
exp['size'] = round(np.sqrt(exp['count']/2))*3
exp.head()

,median,count,size
experience,,,
0,18250,8,6.0
1,10000,53,15.0
2,18000,72,18.0
3,25000,57,15.0
4,25000,70,18.0


In [136]:
source = ColumnDataSource(exp)
p = figure()
p.circle(x='experience', y='median', source=source, size='size')
p.title.text = 'Salario medio de acuerdo a a la experiencia'
p.xaxis.axis_label = 'Experiencia (años)'
p.yaxis.axis_label = 'Salario bruto mensual (MXN)'
p.yaxis.formatter = NumeralTickFormatter(format='$0 a')

hover = HoverTool()
hover.tooltips=[
    ('Experiencia', '@experience años'),
    ('Observaciones', '@count'),
    ('Salario medio', '@median{$0,0}'),
]

p.add_tools(hover)

show(p)

## Comparación por género

In [145]:
# Agrupamos por experiencia y género
gender = df[(df['experience'] > 0)].groupby(["experience", "gender"])["salarymx"].agg(['median','count'])
# Descartamos grupos con menos de 5 observaciones.
gender = gender[(gender['count'] > 4)]
gender.head()

median  count
experience gender               
1          hombre   10000     36
           mujer    16000     15
2          hombre   19000     59
           mujer    15500     12
3          hombre   26750     44

In [148]:
# Groupby deja experience y gender como un multiindex, en lugar de como columnas.
# Así que le damos un reset_index para que los convierta a columnas normales. 
gender = gender.reset_index()
gender.head()

,index,experience,gender,median,count
0,0,1,hombre,10000,36
1,1,1,mujer,16000,15
2,2,2,hombre,19000,59
3,3,2,mujer,15500,12
4,4,3,hombre,26750,44


In [149]:
# Ahora que gender ya es una columna normal, podemos usarla como referencia para crear una columna de color.
gender['color'] = ['#1f77b4' if x =='hombre' else '#e617e6' for x in gender['gender']] 

# También le agregamos el size amortiguado
gender['size'] = round(np.sqrt(gender['count']/2))*3
gender.head()

,index,experience,gender,median,count,color,size
0,0,1,hombre,10000,36,#1f77b4,12.0
1,1,1,mujer,16000,15,#e617e6,9.0
2,2,2,hombre,19000,59,#1f77b4,15.0
3,3,2,mujer,15500,12,#e617e6,6.0
4,4,3,hombre,26750,44,#1f77b4,15.0


In [151]:
src = ColumnDataSource(gender)
p = figure()
p.circle(x='experience', y='median', source=src, size='size', color='color')

hover = HoverTool()
hover.tooltips=[
    ('Género', '@gender'),    
    ('Experiencia', '@experience años'),
    ('Observaciones', '@count'),
    ('Salario medio', '@median{$0,0}'),
]

p.add_tools(hover)

show(p)
